In [1]:
import json
import random
import time
from urllib.parse import urlencode

import requests


def get_sec_uid(username):
    """
    Lấy secuid của người dùng TikTok thông qua username
    """
    try:
        username = username.replace("@", "")
        user_agents = [
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
            "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        ]
        user_agent = random.choice(user_agents)
        headers = {
            "User-Agent": user_agent,
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Connection": "keep-alive",
            "Upgrade-Insecure-Requests": "1",
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Cache-Control": "max-age=0",
        }
        url = f"https://www.tiktok.com/@{username}"
        print(f"Đang lấy thông tin người dùng: {username}")
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            html_content = response.text
            secuid_patterns = ['"secUid":"', '"sec_uid":"', "secUid:", "sec_uid:"]
            secuid = None
            for pattern in secuid_patterns:
                start_idx = html_content.find(pattern)
                if start_idx != -1:
                    start_idx += len(pattern)
                    end_idx = html_content.find('"', start_idx)
                    if end_idx != -1:
                        secuid = html_content[start_idx:end_idx]
                        break
            if secuid:
                print(f"Thành công! SecUID: {secuid}")
                return secuid
            else:
                print("Không tìm thấy secuid trong response")
                return None
        else:
            print(f"Request thất bại với status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Lỗi khi lấy secuid: {str(e)}")
        return None


class TikTokVideoScraper:
    """
    TikTok Video Scraper - Lấy danh sách video của một user TikTok
    """

    def __init__(self):
        self.base_url = "https://www.tiktok.com/api/post/item_list/"
        self.headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0"
            ),
            "Accept": "application/json, text/plain, */*",
            "Accept-Language": "vi-VN,vi;q=0.9,en;q=0.8",
            "Connection": "keep-alive",
            "Referer": "https://www.tiktok.com/",
            "Origin": "https://www.tiktok.com",
        }

    def get_videos_page(self, sec_uid, cursor=0, count=16):
        # Sinh các giá trị động
        import string

        device_id = str(random.randint(7000000000000000000, 7999999999999999999))
        odinId = str(random.randint(7000000000000000000, 7999999999999999999))
        msToken = "".join(
            random.choices(string.ascii_letters + string.digits + "-_", k=128)
        )

        # X-Bogus và X-Gnarly giả lập
        def gen_x_bogus(query_string, user_agent):
            timestamp = int(time.time() * 1000)
            hash_input = f"{query_string}{timestamp}{user_agent}"
            hash_md5 = __import__("hashlib").md5(hash_input.encode()).hexdigest()
            random_chars = "".join(
                random.choices(string.ascii_letters + string.digits, k=16)
            )
            return f"DFSzswVu{random_chars[:4]}AN{hash_md5[:8]}{random_chars[4:]}"

        def gen_x_gnarly(query_string, user_agent):
            timestamp = int(time.time() * 1000)
            hash_input = f"{query_string}{timestamp}{user_agent}"
            hash_value = __import__("hashlib").md5(hash_input.encode()).hexdigest()
            random_chars = "".join(
                random.choices(string.ascii_letters + string.digits, k=20)
            )
            return f"XG_{hash_value[:8]}_{timestamp}_{random_chars}"

        params = {
            "WebIdLastTime": "0",
            "aid": "1988",
            "app_language": "vi-VN",
            "app_name": "tiktok_web",
            "browser_language": "vi",
            "browser_name": "Mozilla",
            "browser_online": "true",
            "browser_platform": "Win32",
            "browser_version": (
                "5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0"
            ),
            "channel": "tiktok_web",
            "clientABVersions": "74014084,74141281",
            "cookie_enabled": "true",
            "count": str(count),
            "coverFormat": "2",
            "cursor": str(cursor),
            "data_collection_enabled": "true",
            "device_id": device_id,
            "device_platform": "web_pc",
            "focus_state": "true",
            "from_page": "user",
            "history_len": "4",
            "is_fullscreen": "false",
            "is_page_visible": "true",
            "language": "vi-VN",
            "locate_item_id": "",
            "needPinnedItemIds": "true",
            "odinId": odinId,
            "os": "windows",
            "post_item_list_request_type": "0",
            "priority_region": "VN",
            "referer": "https://www.tiktok.com/",
            "region": "VN",
            "root_referer": "https://www.tiktok.com/",
            "screen_height": "1080",
            "screen_width": "1920",
            "secUid": sec_uid,
            "tz_name": "Asia/Bangkok",
            "user_is_login": "true",
            "webcast_language": "vi-VN",
            "msToken": msToken,
        }
        query_string = urlencode(params, doseq=True)
        user_agent = self.headers["User-Agent"]
        params["X-Bogus"] = gen_x_bogus(query_string, user_agent)
        params["X-Gnarly"] = gen_x_gnarly(query_string, user_agent)
        headers = self.headers.copy()
        headers["Referer"] = f"https://www.tiktok.com/@user?secUid={sec_uid}"
        try:
            response = requests.get(
                self.base_url, params=params, headers=headers, timeout=15
            )
            if response.status_code == 200:
                return response.json()
            else:
                print(f"❌ API Error: {response.status_code}")
                return None
        except Exception as e:
            print(f"❌ Request failed: {e}")
            return None

    def get_all_videos(self, sec_uid, max_videos=None, delay=2):
        all_videos = []
        cursor = 0
        page = 1
        print(f"🚀 Bắt đầu lấy video cho secUid: {sec_uid}")
        while True:
            print(f"\n📄 Trang {page} - Cursor: {cursor}")
            data = self.get_videos_page(sec_uid, cursor)
            if not data:
                print("❌ Không thể lấy dữ liệu từ API")
                break
            videos = data.get("itemList", [])
            if not videos:
                print("✅ Không có video nào để lấy thêm")
                break
            all_videos.extend(videos)
            print(f"✅ Trang {page}: +{len(videos)} videos, Tổng: {len(all_videos)}")
            if not data.get("hasMore", False):
                print("✅ Đã lấy hết tất cả video")
                break
            cursor = data.get("cursor", cursor)
            if max_videos and len(all_videos) >= max_videos:
                print(f"✅ Đã đạt giới hạn {max_videos} video")
                break
            page += 1
            print(f"⏳ Đang chờ {delay} giây...")
            time.sleep(delay)
        return all_videos

    def save_to_json(self, videos, filename=None):
        import datetime

        if filename is None:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"tiktok_videos_{timestamp}.json"
        try:
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(videos, f, indent=2, ensure_ascii=False)
            print(f"💾 Đã lưu {len(videos)} video vào: {filename}")
            return filename
        except Exception as e:
            print(f"❌ Lỗi khi lưu JSON: {e}")
            return None


if __name__ == "__main__":
    print("🎯 TikTok Video Scraper")
    print("=" * 60)
    username = input("Nhập username TikTok (không cần @): ").strip()
    sec_uid = get_sec_uid(username)
    if not sec_uid:
        print("❌ Không lấy được secUid, kiểm tra lại username!")
    else:
        try:
            max_videos = input(
                "Số lượng video tối đa (Enter = không giới hạn): "
            ).strip()
            max_videos = int(max_videos) if max_videos else None
        except ValueError:
            max_videos = None
        scraper = TikTokVideoScraper()
        print("\n🚀 Bắt đầu lấy video...")
        videos = scraper.get_all_videos(sec_uid, max_videos)
        if not videos:
            print("❌ Không lấy được video nào")
        else:
            print(f"\n✅ Hoàn thành! Đã lấy {len(videos)} video")
            scraper.save_to_json(videos)


🎯 TikTok Video Scraper
Đang lấy thông tin người dùng: lekimngan0101
Thành công! SecUID: MS4wLjABAAAALCH_Y36HZTtfv4oXaa6FMqg0KWRwZUqjS9KHWmXBkWxKjSeFKy2Gpb-bvIwDXL3j

🚀 Bắt đầu lấy video...
🚀 Bắt đầu lấy video cho secUid: MS4wLjABAAAALCH_Y36HZTtfv4oXaa6FMqg0KWRwZUqjS9KHWmXBkWxKjSeFKy2Gpb-bvIwDXL3j

📄 Trang 1 - Cursor: 0
✅ Trang 1: +18 videos, Tổng: 18
⏳ Đang chờ 2 giây...

📄 Trang 2 - Cursor: 1755221129607
✅ Trang 2: +16 videos, Tổng: 34
⏳ Đang chờ 2 giây...

📄 Trang 3 - Cursor: 1754993959972
✅ Trang 3: +16 videos, Tổng: 50
⏳ Đang chờ 2 giây...

📄 Trang 4 - Cursor: 1754814724659


KeyboardInterrupt: 

In [3]:
!pip install pyppeteer

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import asyncio
from pyppeteer import launch
def main():
    import time

    async def main():
        browser = await launch(headless=True)
        page = await browser.newPage()
        await page.setUserAgent(
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        )
        url = "https://www.tiktok.com/@aiacademygb/video/7478554860850253064"
        await page.goto(url)    
        content = await page.content()
        await browser.close()
        return content
    
    content = asyncio.get_event_loop().run_until_complete(main())
    print(content)
    
